## Import libraries

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import cv2
import albumentations as A

In [3]:
import random
random.seed(7)

In [4]:


BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)


## Set image and label directory

In [5]:
import os
IMAGE_DIR = 'images'
LABEL_DIR = 'labels'
img_files = os.listdir(IMAGE_DIR)
label_files = os.listdir(LABEL_DIR)

In [6]:
def extract_labels(label_path):
  bboxes = []
  category_ids = []

  # Read from label text file
  with open(label_path, 'r') as file:
    lines = file.readlines()

  breakpoint()

  for line in lines:

    parts = line.strip().split()
    print(parts)
    class_num = parts[0]
    bbox_coordinates = [float(part) for part in parts[1:]]

    category_ids.append(class_num)
    bboxes.append(bbox_coordinates) 
  
  return category_ids, bboxes

In [7]:
def augment_image(img, transform, bboxes, category_ids, file_name):
  images_list = []          # List of string
  saved_bboxes = []         # List of list
  saved_category_ids = []   # List of list
  out_img_paths = []        # List of string
  out_label_paths = []      # List of string

  # for thrice the dataset: 2 augmentations for a single image
  for i in range(2):
    
    outfile = f'{file_name}-aug{i}'
    out_img_path = os.path.join(IMAGE_DIR, f'{outfile}.jpg')
    out_label_path = os.path.join(LABEL_DIR, f'{outfile}.txt')

    transformed = transform(image=img, bboxes=bboxes, category_ids=category_ids)

    if len(transformed["bboxes"]) == 0:
      continue

    images_list.append(transformed["image"])
    saved_bboxes.append(transformed["bboxes"])
    saved_category_ids.append(transformed["category_ids"])
    out_img_paths.append(out_img_path)
    out_label_paths.append(out_label_path)
  
  return images_list, saved_bboxes, saved_category_ids, out_img_paths, out_label_paths

## Main script

In [8]:
for img_file in img_files:

  # Split the filename and extension
  file_name, file_extension = os.path.splitext(img_file)
  print(file_name)

  img_path = os.path.join(IMAGE_DIR, f'{file_name}.jpg')
  label_path = os.path.join(LABEL_DIR, f'{file_name}.txt')

  # Read image & convert to RGB format
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

  # Extract labels from text file
  category_ids, bboxes = extract_labels(label_path)

  # Composition of transformations
  transform = A.Compose(
  [
    A.HorizontalFlip(p=0.8),
    A.VerticalFlip(p=0.2),
    A.OneOf([
      A.Blur(blur_limit=5, p=0.5),
      A.MotionBlur(blur_limit=5, p=0.8)
    ], p=1.0)
  ], bbox_params=A.BboxParams(format='yolo',  min_visibility=0.33, min_area=3600., label_fields=['category_ids'])
) 
  

  # Get augmented images for a single image
  images_list, saved_bboxes, saved_category_ids, out_img_paths, out_label_paths = augment_image(img, transform, bboxes, category_ids, file_name) 

  # Write results to file
  # Write the augmented image to images directory
  for img, out_img_path in zip(images_list, out_img_paths):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_img_path, img)
  

  # Write the labels to label text file and put into labels directory
  for list_category_ids, list_saved_bboxes, out_label_path in zip(saved_category_ids, saved_bboxes, out_label_paths):
    with open(out_label_path, 'a') as txt_file:
      
      for aug_category_id, aug_bbox in zip(list_category_ids, list_saved_bboxes):
        labels = []
        labels.append(str(aug_category_id))
        bbox_info = [str(info) for info in aug_bbox]
        labels += bbox_info
      
        label_txt = ' '.join(labels)

        txt_file.write(label_txt + '\n')

6b374cd3-april18-17
['1', '0.788137755102041', '0.6926020408163265', '0.4237244897959183', '0.6147959183673466']
['2', '0.8447704081632648', '0.40306122448979564', '0.3104591836734689', '0.43469387755101985']
61850597-april18-25
['0', '0.10867346938775514', '0.5385204081632652', '0.21734693877551028', '0.4770408163265302']
5d869984-april18-28
['1', '0.4790816326530613', '0.5158163265306123', '0.20204081632653065', '0.2846938775510204']
['2', '0.9197704081632649', '0.7477040816326533', '0.16045918367346992', '0.4301020408163269']
a4e9b82b-April16-38
['0', '0.1796979865771812', '0.6986577181208052', '0.3593959731543624', '0.6026845637583887']
bd463858-April16-14
['2', '0.8049744897959183', '0.7058673469387757', '0.39005102040816325', '0.5403061224489799']
['1', '0.1346938775510204', '0.4477040816326532', '0.2693877551020408', '0.43622448979591844']
d2e4295d-April16-9
['0', '0.14961734693877551', '0.7359693877551018', '0.29923469387755103', '0.45561224489795854']
a4b6f96b-april18-2
['2', 